In [2]:
import os
import openai


In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")


In [4]:
def test_prompt_openai(prompt, suppress=False, model='text-davinci-003', **kwargs):
    response = openai.Completion.create(
      model=model,
      prompt=prompt,
      max_tokens=256,
      **kwargs
    )
    answer = response.choices[0].text
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{answer}')
    else:
        return answer


In [13]:
NAMESPACE = 'default'  # pinecone namespace

In [14]:
import requests
import json

def get_best_result_from_pinecone(query, namespace=NAMESPACE):
    payload = json.dumps({
      "num_results": 2,
      "query": query,
      "re_ranking_strategy": "none",
      "namespace": namespace
    })

    response = requests.post(
        "https://information-retrieval-hiaa.onrender.com/document/retrieve", 
        data=payload
    )

    return response.json()['documents'][0]


In [15]:
query = "What are fixed costs?"

best_result = get_best_result_from_pinecone(query)
    
PROMPT = f"""
Answer the question using the context.

Context: {best_result['text']}
Query: {query}
Answer:""".strip()

test_prompt_openai(PROMPT)

PROMPT:
------
Answer the question using the context.

Context: In economics, fixed costs, indirect costs or overheads are business expenses that are not dependent on the level of goods or services produced by the business. They tend to be time-related, such as salaries or rents being paid per month, and are often referred to as overhead costs. This is in contrast to variable costs, which are volume-related (and are paid per quantity produced). For a simple example, such as a bakery, the monthly rent for the baking facilities, and the monthly payments for the security system and basic phone line are fixed costs, as they do not change according to how much bread the bakery produces and sells. On the other hands, the wage costs of the bakery are variable, as the bakery will have to hire more workers if the production of bread increases. The relation between fixed cost and variable cost can be modelled by an analytical formula.
Query: What are fixed costs?
Answer:
------
RESPONSE
------
 

In [16]:
query = "How old is Obama?"

best_result = get_best_result_from_pinecone(query)
    
PROMPT = f"""
Answer the question using the context.

Context: {best_result['text']}
Query: {query}
Answer:""".strip()

test_prompt_openai(PROMPT)

PROMPT:
------
Answer the question using the context.

Context: In November 2008, the show's post-election day telecast garnered the biggest audience in the show's history at 6.2 million in total viewers, becoming the week's most-watched program in daytime television. It was surpassed on July 29, 2010, during which former President Barack Obama first appeared as a guest on The View, which garnered a total of 6.6 million viewers. In 2013, the show was reported to be averaging 3.1 million daily viewers, which outpaced rival talk show The Talk.
Query: How old is Obama?
Answer:
------
RESPONSE
------
 Obama is 58 years old (as of October 2020).


In [37]:
# Asking it to not answer if the context isn't enough.
query = "How old is Obama?"

best_result = get_best_result_from_pinecone(query)

PROMPT = f"""
Only using the following context, answer the question. If you cannot answer using the context, say 'I don't know. Use this format:

Context: (context)
Query: (natural language query)
Answer: (answer)

Context: {best_result['text']}
Query: {query}
Answer:""".strip()

test_prompt_openai(PROMPT)


PROMPT:
------
Only using the following context, answer the question. If you cannot answer using the context, say 'I don't know. Use this format:

Context: (context)
Query: (natural language query)
Answer: (answer)

Context: In November 2008, the show's post-election day telecast garnered the biggest audience in the show's history at 6.2 million in total viewers, becoming the week's most-watched program in daytime television. It was surpassed on July 29, 2010, during which former President Barack Obama first appeared as a guest on The View, which garnered a total of 6.6 million viewers. In 2013, the show was reported to be averaging 3.1 million daily viewers, which outpaced rival talk show The Talk.
Query: How old is Obama?
Answer:
------
RESPONSE
------
 I don't know.


In [47]:
# use some chain of thought prompting to make it think through the reasoning.
def gen_Q_A(query, qa_engine='openai'):
    best_result = get_best_result_from_pinecone(query)
    
    PROMPT = f"""
Only using the following context, answer the question.

Context: (context)
Query: (natural language query)
Answer: (answer)
Reasoning: (the reasoning as to how the query is answerable or not using the context)

Context: {best_result['text']}
Query: {query}
Answer:""".strip()
    
    if qa_engine == 'openai':
        return test_prompt_openai(PROMPT)
    elif qa_engine == 'cohere':
        return test_prompt_cohere(PROMPT)
        

In [50]:
gen_Q_A('How old is Obama?', qa_engine='openai')

PROMPT:
------
Only using the following context, answer the question.

Context: (context)
Query: (natural language query)
Answer: (answer)
Reasoning: (the reasoning as to how the query is answerable or not using the context)

Context: In November 2008, the show's post-election day telecast garnered the biggest audience in the show's history at 6.2 million in total viewers, becoming the week's most-watched program in daytime television. It was surpassed on July 29, 2010, during which former President Barack Obama first appeared as a guest on The View, which garnered a total of 6.6 million viewers. In 2013, the show was reported to be averaging 3.1 million daily viewers, which outpaced rival talk show The Talk.
Query: How old is Obama?
Answer:
------
RESPONSE
------
 Unanswerable
Reasoning: The context provided does not contain information regarding Obama's age.


In [41]:
gen_Q_A('how many innings in a baseball game?', qa_engine='openai')

PROMPT:
------
Only using the following context, answer the question. If you cannot answer using the context, say 'I don't know. Use this format

Context: (context)
Query: (natural language query)
Reasoning: (the reasoning as to how the query is answerable or not using the context)
Answer: (answer)

Context: Ordinarily, a baseball game consists of nine innings (in softball and high school baseball games there are typically seven innings; in Little League Baseball, six), each of which is divided into halves: the visiting team bats first, after which the home team takes its turn at bat. However, if the score remains tied at the end of the regulation number of complete innings, the rules provide that ``play shall continue until (1) the visiting team has scored more total runs than the home team at the end of a completed inning; or (2) the home team scores the winning run in an uncompleted inning.'' (Since the home team bats second, condition (2) implies that the visiting team will not hav

In [42]:
gen_Q_A('How old is Obama?', qa_engine='openai')

PROMPT:
------
Only using the following context, answer the question. If you cannot answer using the context, say 'I don't know. Use this format

Context: (context)
Query: (natural language query)
Reasoning: (the reasoning as to how the query is answerable or not using the context)
Answer: (answer)

Context: In November 2008, the show's post-election day telecast garnered the biggest audience in the show's history at 6.2 million in total viewers, becoming the week's most-watched program in daytime television. It was surpassed on July 29, 2010, during which former President Barack Obama first appeared as a guest on The View, which garnered a total of 6.6 million viewers. In 2013, the show was reported to be averaging 3.1 million daily viewers, which outpaced rival talk show The Talk.
Query: How old is Obama?
Reasoning:
------
RESPONSE
------
 The context does not contain sufficient information to answer this query.
Answer: I don't know.


In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

sequence_to_classify = "All of this is your fault for being Jewish. You shouldn't be allowed to have an account on this website in the first place. If you want help, you'll need to take care of this problem yourself."

candidate_labels = ['racist', 'anti-semitic', 'sexist']

classifier(sequence_to_classify, candidate_labels, multi_label=True)  # Assuming there can be multiple answers


# then the "rude" AI wasn't that bad
classifier(
    'Do you have your login information? Because if not, then there is nothing I can do for you.', 
    candidate_labels, multi_label=True)



# Retrieval Augmented Generation with GPT-4

In [ ]:
# TODO add output bias

In [31]:
SYSTEM_PROMPT = '''
You are a helpful Q/A bot that can only use reference material from a knowledge base.
If a user asks anything that is not "from the knowledge base", say that you cannot answer.
'''.strip()

class ChatbotGPT():
    def __init__(self, system_prompt, threshold=.8):
        self.conversation = [{'role': 'system', 'content': system_prompt}]
        self.threshold = threshold

    def display_conversation(self):
        '''display the conversation in a pretty format denoting the system, user and assistant differently'''
        for turn in self.conversation:
            role = turn['role']
            content = turn['content']
            if role == 'system':
                print(f'System: {content}')
            elif role == 'user':
                print(f'User: {content}')
            elif role == 'assistant':
                print(f'Assistant: {content}')
            print('------------')

    def user_turn(self, message):
        self.conversation.append({"role": "user", "content": message})
        best_result = get_best_result_from_pinecone(message)
        print(best_result)
        if best_result['score'] >= self.threshold:
            # Add to the context to the system prompt
            self.conversation[0]['content']+=f'\n\nFrom the knowledge base: "{best_result["text"]}"'

        chatgpt_response = openai.ChatCompletion.create(
            model='gpt-4',
            temperature=0,
            messages=self.conversation
        ).choices[0].message.content.strip()
        self.conversation.append({'role': 'assistant', 'content': chatgpt_response})
        return self.conversation[-1]


In [32]:
c = ChatbotGPT(system_prompt=SYSTEM_PROMPT)

c.user_turn('what are fixed costs?')

{'text': 'In economics, fixed costs, indirect costs or overheads are business expenses that are not dependent on the level of goods or services produced by the business. They tend to be time-related, such as salaries or rents being paid per month, and are often referred to as overhead costs. This is in contrast to variable costs, which are volume-related (and are paid per quantity produced). For a simple example, such as a bakery, the monthly rent for the baking facilities, and the monthly payments for the security system and basic phone line are fixed costs, as they do not change according to how much bread the bakery produces and sells. On the other hands, the wage costs of the bakery are variable, as the bakery will have to hire more workers if the production of bread increases. The relation between fixed cost and variable cost can be modelled by an analytical formula.', 'date_uploaded': '2023-07-31T13:18:21.538702', 'score': 0.891583443, 'id': '57a0103e5716168be7498b4531b21d07'}


{'role': 'assistant',
 'content': 'Fixed costs, in economics, are business expenses that are not dependent on the level of goods or services produced by the business. They tend to be time-related, such as salaries or rents being paid per month, and are often referred to as overhead costs. For example, in a bakery, the monthly rent for the baking facilities, and the monthly payments for the security system and basic phone line are fixed costs, as they do not change according to how much bread the bakery produces and sells.'}

In [33]:
c.user_turn('How old is Obama?')

{'text': "In November 2008, the show's post-election day telecast garnered the biggest audience in the show's history at 6.2 million in total viewers, becoming the week's most-watched program in daytime television. It was surpassed on July 29, 2010, during which former President Barack Obama first appeared as a guest on The View, which garnered a total of 6.6 million viewers. In 2013, the show was reported to be averaging 3.1 million daily viewers, which outpaced rival talk show The Talk.", 'date_uploaded': '2023-07-31T13:17:59.802219', 'score': 0.785162151, 'id': 'be0b9cb01bae3d7fa13839b85c10079c'}


{'role': 'assistant',
 'content': "I'm sorry, but I can't provide the answer to that question because it's not in my knowledge base."}

In [34]:
c.user_turn('how many innings are in a game of baseball?')

{'text': "Ordinarily, a baseball game consists of nine innings (in softball and high school baseball games there are typically seven innings; in Little League Baseball, six), each of which is divided into halves: the visiting team bats first, after which the home team takes its turn at bat. However, if the score remains tied at the end of the regulation number of complete innings, the rules provide that ``play shall continue until (1) the visiting team has scored more total runs than the home team at the end of a completed inning; or (2) the home team scores the winning run in an uncompleted inning.'' (Since the home team bats second, condition (2) implies that the visiting team will not have the opportunity to score more runs before the end of the inning.)", 'date_uploaded': '2023-07-31T13:18:08.453184', 'score': 0.91251117, 'id': 'b2f88d00f95a155e6092e181a763ad03'}


{'role': 'assistant',
 'content': 'Ordinarily, a baseball game consists of nine innings. Each of these innings is divided into halves: the visiting team bats first, after which the home team takes its turn at bat. However, if the score remains tied at the end of the regulation number of complete innings, the game can continue until either the visiting team has scored more total runs than the home team at the end of a completed inning, or the home team scores the winning run in an uncompleted inning.'}

In [35]:
c.user_turn('thanks so much!')

{'text': "Susie moves on into another, larger part of heaven, occasionally watching earthbound events. Lindsey and Samuel have a daughter together named Abigail Suzanne. While stalking a young woman in New Hampshire, Harvey is hit on the shoulder by an icicle and falls to his death down a snow-covered slope into the ravine below. At the end of the novel, a Norristown couple finds Susie's charm bracelet but don't realize its significance, and Susie closes the story by wishing the reader ``a long and happy life''.", 'date_uploaded': '2023-07-31T13:18:14.183838', 'score': 0.736579299, 'id': '8ae1783aed03559f78a886c77e8001ff'}


{'role': 'assistant',
 'content': "You're welcome! If you have any other questions, feel free to ask."}

In [36]:
c.display_conversation()

System: You are a helpful Q/A bot that can only reference material from a knowledge base.
If a user asks anything that is not "from the knowledge base", say that you cannot answer.

From the knowledge base: "In economics, fixed costs, indirect costs or overheads are business expenses that are not dependent on the level of goods or services produced by the business. They tend to be time-related, such as salaries or rents being paid per month, and are often referred to as overhead costs. This is in contrast to variable costs, which are volume-related (and are paid per quantity produced). For a simple example, such as a bakery, the monthly rent for the baking facilities, and the monthly payments for the security system and basic phone line are fixed costs, as they do not change according to how much bread the bakery produces and sells. On the other hands, the wage costs of the bakery are variable, as the bakery will have to hire more workers if the production of bread increases. The relat